In [1]:
import numpy as np
np.set_printoptions(precision=3)

In [2]:
class Person:
    def __init__(self, leverage, pledge):
        self.leverage = leverage
        self.pledge = pledge

In [3]:
class Pool:
    def __init__(self, target):
        self.target = target
        self.people = []
        self.pot = 0
        self.epsilon = 0.0001
        self.alpha = 0.1
    
    def get_pledges(self):
        return [person.pledge for person in self.people]
    
    def get_payouts(self):
        return [person.payout for person in self.people]
    
    def get_saturation(self):
        pledges = self.get_pledges()
        radiuses = np.sqrt(pledges)
        slope = sum(radiuses) / self.target
        angle = np.arctan(slope)
        saturation = angle / (np.pi / 2)
        return saturation
    
    def update_pot(self):
        saturation = self.get_saturation()
        # update payouts
        for person in self.people:
            person.payout = person.pledge * saturation
        # update pot
        self.pot = sum(self.get_payouts())
    
    def update_pledges(self):
#         person = self.people[person_num]
        for person in self.people:
            self.update_pot()
            old_pot = self.pot
            old_payout = person.payout

            # this change from updating epsilon is negligible
            person.pledge += self.epsilon
            self.update_pot()
            new_pot = self.pot
            new_payout = person.payout
            leverage = (new_pot - old_pot) / (new_payout - old_payout)
#             print(leverage)
            gradient = (leverage - person.leverage) * self.alpha
            gradient = np.clip(gradient, -self.alpha, self.alpha)
            person.pledge += gradient
    
        
pool = Pool(target=10)
leverages = [1.5, 1.5, 1.5, 1.5, 1.5, 1.2, 1.2, 1.2, 1.2, 1.2]

for leverage in leverages:
    pool.people.append(Person(leverage, 0))

pool.people[0].pledge = 1
pool.people[1].pledge = 1
pool.people[2].pledge = 1
pool.people[3].pledge = 1
pool.people[4].pledge = 1
pool.people[5].pledge = 1
pool.people[6].pledge = 0
pool.people[7].pledge = 0
pool.people[8].pledge = 0
pool.people[9].pledge = 0.1

In [4]:
pool.update_pledges()

pool.update_pot()
print("pot:        ", pool.pot)
print("saturation: ", pool.get_saturation())

pool.get_pledges()

pot:         2.5751651080932567
saturation:  0.402732953634935


[0.9805252340944199,
 0.9804697098543417,
 0.9804137337202606,
 0.9803573005564846,
 0.9803004051545005,
 1.0102430422173485,
 0.10010000000000001,
 0.10010000000000001,
 0.10010000000000001,
 0.18161554370466973]